<h1>Segmenting and Clustering Neighborhoods in Toronto (Part 3)</h1>

<h4>Import libraries</h4>

In [34]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.48 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  37.25 MB/s
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [19]:
#Extract the table from the website
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')
table = soup.find('table', class_='wikitable')

#Extract the elements from the table
postcode = []
borough = []
neighborhood = []

for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        postcode.append(cells[0].find(text=True).rstrip('\n'))
        borough.append(cells[1].find(text=True).rstrip('\n'))
        neighborhood.append(cells[2].find(text=True).rstrip('\n'))
        
#Write the elements in a dataframe
df = pd.DataFrame()
df['Postal Code'] = postcode
df['Borough'] = borough
df['Neighborhood'] = neighborhood

#Remove rows where Borough = Not Assigned
df = df[df["Borough"] != "Not assigned"]

#Group neighbordhoods with same postcode
for i in range(1,df.shape[0]):
    if (df.iloc[i,0] == df.iloc[i-1,0])==True:
        df.iloc[i-1,0] = 'Not assigned'
        df.iloc[i,2] = df.iloc[i-1,2] + ", " + df.iloc[i,2]
           
df = df[df["Postal Code"] != "Not assigned"]

#Assign Borough name to Neighborhood if Neighborhood = Not assigned
for i in range(1,df.shape[0]):
    if (df.iloc[i,2] == "Not assigned")==True:
        df.iloc[i,2] = df.iloc[i,1]
        
df = df.sort_values(by = ["Postal Code"])

df = df.reset_index(drop = True)

df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [2]:
#Get geographical coordinates from url
geo_coordinates = pd.read_csv('http://cocl.us/Geospatial_data')

geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
#Merge the data frames
neighborhoods = pd.merge(df, geo_coordinates, how='inner', on='Postal Code')

neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h4>Create a map of Scarborough (one of the bouroughs in Toronto) with neighborhoods superimposed on top</h4>

In [22]:
#Restrict dataframe to neighborhoods in Scarborough
Scarborough_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough']
Scarborough_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [78]:
# create map of Scarborough using latitude and longitude values

#Let's start with the average lat and long in Scarborough
latitude = Scarborough_data['Latitude'].mean()
longitude = Scarborough_data['Longitude'].mean()
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=10, width=300,height=300)

#Add markers to map
for lat, lng, borough, neighborhood in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

<h4>Define function to get nearby venues</h4>

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h4>Run the above function on each neighborhood and create a new dataframe called Scarborough_venues</h4>

In [ ]:
# The code was removed by Watson Studio for sharing.

In [24]:
#First define input
CLIENT_ID = '5FLE230IRIWTXYA3WZC2B3B50NPG2OL1G3NLNWZPLCHO2MEA' # your Foursquare ID
CLIENT_SECRET = 'R3M4RHUSFKILVOFC00H52RT32DNENEX2WIYQZUXEBWVD4WJE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

#Then call the function
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


In [9]:
#Let's check the size of the resulting dataframe
print(Scarborough_venues.shape)
Scarborough_venues.head()

(91, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [10]:
#Let's check how many venues were returned for each neighborhood
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",9,9,9,9,9,9
"Cliffcrest, Cliffside, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre, Wexford Heights",8,8,8,8,8,8
"East Birchmount Park, Ionview, Kennedy Park",9,9,9,9,9,9


<h4>Analyze each neighborhood</h4>

In [11]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

#Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_grouped

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.250000,0.000,...,0.000000,0.000000,0.000,0.250000,0.0,0.0,0.000000,0.000000,0.000000,0.000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,...,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000
2,"Birch Cliff, Cliffside West",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,...,0.000000,0.000000,0.000,0.250000,0.0,0.0,0.000000,0.000000,0.000000,0.000
3,Cedarbrae,0.000,0.000000,0.142857,0.000,0.142857,0.142857,0.0,0.000000,0.000,...,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.142857,0.000000,0.000000,0.000
4,"Clairlea, Golden Mile, Oakridge",0.000,0.000000,0.000000,0.000,0.200000,0.000000,0.0,0.000000,0.000,...,0.000000,0.000000,0.000,0.000000,0.0,0.1,0.000000,0.000000,0.000000,0.000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,...,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.111111,0.000000,0.000000,0.000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.000,0.333333,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,...,0.000000,0.000000,0.000,0.333333,0.0,0.0,0.000000,0.000000,0.000000,0.000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.125,...,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.125000,0.000000,0.125
8,"East Birchmount Park, Ionview, Kennedy Park",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,...,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.111111,0.000
9,"Guildwood, Morningside, West Hill",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.166667,0.000,...,0.166667,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000


<h4>Let's create a new dataframe and display the top 10 venues for each neighborhood</h4>

In [12]:
#First, let's write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

#Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#Create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Caribbean Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Hakka Restaurant,Coffee Shop,General Entertainment,Food Truck
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Park,Food Truck,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Clothing Store,Fried Chicken Joint
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Noodle House,Fast Food Restaurant,Italian Restaurant,Chinese Restaurant,Pharmacy,Fried Chicken Joint,Thai Restaurant,Bank,Clothing Store
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Skating Rink,Motel,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Thrift / Vintage Store,Latin American Restaurant,Brewery,Pet Store,Food Truck,Fast Food Restaurant,Electronics Store
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Hobby Shop,Bus Station,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Train Station,Auto Garage,Grocery Store
9,"Guildwood, Morningside, West Hill",Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Food Truck,Fast Food Restaurant


<h4>Cluster Neighborhoods</h4>

In [73]:
#Run k-means to cluster the neighborhoods
# set number of clusters
kclusters = 3

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [74]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
#Add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

Scarborough_merged = Scarborough_data

#Merge Scarborough_grouped with Scarborough_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scarborough_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Electronics Store,Discount Store,Department Store,Convenience Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1.0,Bar,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Food Truck,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Hakka Restaurant,Coffee Shop,General Entertainment,Food Truck
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Smoke Shop,Playground,Vietnamese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0.0,Discount Store,Hobby Shop,Bus Station,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Train Station,Auto Garage,Grocery Store
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0.0,Bakery,Bus Line,Park,Food Truck,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Clothing Store,Fried Chicken Joint
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0.0,American Restaurant,Skating Rink,Motel,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0.0,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store


In [75]:
#Remove rows where Cluster is NaN
Scarborough_merged = Scarborough_merged[Scarborough_merged["Neighborhood"] != "Upper Rouge"]

#Cast cluster as int
Scarborough_merged["Cluster Labels"] = Scarborough_merged["Cluster Labels"].astype(int)
Scarborough_merged

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Electronics Store,Discount Store,Department Store,Convenience Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,Bar,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Food Truck,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Hakka Restaurant,Coffee Shop,General Entertainment,Food Truck
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Smoke Shop,Playground,Vietnamese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,Discount Store,Hobby Shop,Bus Station,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Train Station,Auto Garage,Grocery Store
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Bakery,Bus Line,Park,Food Truck,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Clothing Store,Fried Chicken Joint
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0,American Restaurant,Skating Rink,Motel,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store


In [76]:
#Finally, let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10, width=300,height=300)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighborhood'], Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4>Conclusions</h4>
I replicated the clustering analysis that was initially done for Manathan on a specific borough of Toronto: Scarborough.
The results suggest that two neighborhoods are quite different from the others: M1B and M1C.
However, I find the results difficult to interpret in depth due to the large number of categories of venues that are taken into account.
It could be interesting to group categories of venues and to replicate the exercise with this new basis.